In [1]:
import keras
import tensorflow as tf

# Memory Pre-configuration
config = tf.compat.v1.ConfigProto(
    gpu_options=tf.compat.v1.GPUOptions(
        per_process_gpu_memory_fraction=0.9,
        allow_growth = True
    )
    # device_count = {'GPU': 1}
)
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

ConvNet이라고 불리는 합성곱 신경망 (Convolutional Neural Network)를 소개합니다. 거의 대부분 컴퓨터 비전 (Computer Vision)에 사용됩니다.

##### 간단한 컨브넷 만들기

In [2]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (28, 28, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation = 'relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation = 'relu'))

컨브넷이 `(image_height, image_width, image_channels)` 크기의 입력 텐서를 사용한다는 점이 중요합니다. 이 예제에서는 MNIST 이미지 포맷인 (28, 28, 1) 크기의 입력을 처리하도록 컨브넷을 설정하였습니다. 이 때문에 첫번째 층의 매개변수로 `input_shape = (28, 28, 1)`을 전달했습니다.

In [3]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


`Conv2D`와 `MaxPooling2D` 층의 출력은 *(height, width, channels)* 크기의 3D 텐서입니다. 높이와 너비 차원은 네트워크가 깊어질수록 작아지는 경향이 있습니다. 채널의 수는 `Conv2D`층에 전달된 첫 번째 매개변수에 의해 조절됩니다.

다음 단계에서 마지막 층의 출력 텐서를 완전 연결 네트워크에 주입합니다. 이 네트워크는 Dense층을 쌓은 분류기입니다.
* 분류기는 1D 벡터를 처리하는데
* 이전 층의 출력이 3D 텐서입니다.
* 먼저 3D 출력을 1D 텐서로 펼쳐야합니다.

##### 컨브넷 위에 분류기 추가하기

In [4]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation = 'relu'))
model.add(layers.Dense(10, activation = 'softmax'))

In [5]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 3, 3, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 576)               0         
_________________________________________________________________
dense (Dense)                (None, 64)                3

여기에서 볼 수 있듯 (3, 3, 64) 출력이 (576, ) 크기의 벡터로 펼쳐진 이후 Dense 층으로 주입되었습니다.

##### MNIST 이미지에 컨브넷 훈련하기

In [6]:
from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32')/255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32')/255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

model.compile(optimizer = 'rmsprop',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])

model.fit(train_images, train_labels, epochs = 5, batch_size = 64)

Epoch 1/5
938/938 [==============================] - 4s 3ms/step - loss: 0.3891 - accuracy: 0.8743
Epoch 2/5
938/938 [==============================] - 2s 2ms/step - loss: 0.0524 - accuracy: 0.9837
Epoch 3/5
938/938 [==============================] - 2s 2ms/step - loss: 0.0334 - accuracy: 0.9897
Epoch 4/5
938/938 [==============================] - 2s 2ms/step - loss: 0.0249 - accuracy: 0.9918
Epoch 5/5
938/938 [==============================] - 2s 2ms/step - loss: 0.0180 - accuracy: 0.9949


In [7]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
test_acc

313/313 [==============================] - 1s 2ms/step - loss: 0.0345 - accuracy: 0.9898


0.989799976348877

이전에 만들었던 완전 연결 네트워크는 98%의 테스트 정확도를 얻은 반면에 기본적인 CNN은 99.2%의 테스트 정확도를 얻었습니다. 왜 간단한 CNN이 더 잘 작동할까요? Conv2D와 MaxPooling2D 층에 대해서 알아봅시다.

### 합성곱 연산

완전 연결 층과 합성곱 층 사이의 근본적인 차이는 다음과 같습니다. Dense 층은 입력 특성 공간에 있는 전역 패턴을 학습하지만 합성곱 층은 지역 패턴을 학습합니다. 이미지일 경우 작은 2D Window로 입력에서 패턴을 찾습니다. 앞의 예시에서는 윈도우는 3x3이였습니다.

<br></br>
![](../images/5-1-cnn1.png)

<br></br>

이 핵심 특징은 컨브넷에 두 가지 흥미로운 성질을 제공합니다.

1. **학습된 패턴은 평행 이동 불변성 *Translation Invariant*을 가집니다.** 컨브넷이 이미지의 오른쪽 아래 모서리에서 어떤 패턴을 학습했다면 다른 곳 (예를 들면 왼쪽 위 모서리) 에서도 이 패턴을 인식할 수 있습니다. 완전 연결 네트워크는 새로운 위치에 나타난 것은 새로운 패턴으로 학습해야 합니다. 이런 성질은 컨브넷이 이미지를 효율적으로 처리하게 만들어 줍니다. 적은 수의 훈련 샘플을 사용해서 일반화 능력을 가진 표현을 학습할 수 있습니다.

2. **컨브넷은 패턴의 공간적 계층 구조를 학습할 수 있습니다.** 첫번째 합성곱 층이 에지 같은 작은 지역 패턴을 학습합니다. 두번째 층은 첫번째 층의 특성으로 구성된 더 큰 패턴을 학습합니다. 이런 방식을 사용하여 컨브넷은 매우 복잡하고 추상적인 시각적 개념을 효과적ㅈ으로 학습할 수 있습니다.

<br></br>

![](../images/5-1-cnn2.png)
<br></br>

합성곱 엽산은 **특성 맵 (feature map)**이라고 부르는 3D 텐서에 적용됩니다. 이 텐서는 2개의 공간 축 (*높이*와 *너비*)와 깊이 축 (*채널* 축)으로 구성됩니다. RGB 이미지는 3개의 컬러 채널 (Red, Green, Blue)을 가지므로 깊이 축의 차원이 *3*이 됩니다. MNIST 숫자처럼 흑백 이미지는 깊이 축의 차원이 *1*입니다. 합성곱 연산은 입력 특성 맵에서 작은 패치 (patch)들을 추출하고 이런 모든 패치에서 같은 변환을 적용하여 출력 특성 맵(output feature map)을 만듭니다.

출력 특성 맵도 높이와 너비를 가진 3D 텐서입니다. 출력 텐서의 깊이는 층의 매개변수로 결정됩니다. 이렇게 되면 깊이 축의 채널은 더 이상 RGB 입력처럼 특정 컬러를 의미하지 않습니다. 대신 일종의 **필터 (filter)**를 의미합니다. 필터는 입력 데이터의 어떤 특성을 인코딩합니다.

MNIST 예제에서는 첫번째 합성곱 층이 (28, 28, 1) 크기의 특성 맵을 입력으로 받아 (26, 26, 32) 크기의 특성 맵을 출력합니다. 즉 입력에 대해 32개의 필터를 적용합니다. 32개의 출력 채널은 각각 26x26 크기의 배열 값을 가집니다. 이 값은 입력에 대한 필터의 **응답 맵 (response map)**입니다. 입력의 각 위치에서 필터 패턴에 대한 응답을 나타냅니다. **특성 맵**이란 깊이 축에 있는 각 차원은 하나의 **특성** (또는 필터)이고, 2D 텐서 output[:,:,n]은 입력에 대한 이 필터 응답을 나타내는 2D 공간상의 **맵**입니다.

<br></br>

![](../images/5-1-cnn3.png)
<br></br>

합성곱은 핵심적인 2개의 파라미터로 정의됩니다.

1. 입력으로부터 뽑아낼 패치의 크기: 전형적으로 3x3, 5x5 크기를 사용합니다.
2. 특성 맵의 출력 깊이: 합성곱으로 계산할 필터의 수입니다. 이 예제에서는 32에서 시작하여 64에서 끝났습니다

3D 입력 특성 맵 위를 3x3 또는 5x5 크기의 윈도우가 **슬라이딩 (sliding)**하면서 모든 위치에서 3D 특성 패치인 (window_height, window_width, input_depth)를 추출하는 방식으로 합성곱이 작동합니다. 이런 3D 패치는 (output_depth,) 크기의 1D 벡터로 변환됩니다. 이 과정은 **합성곱 커널 (convolutional kernel)**이라고 불리는 하나의 학습된 가중치 행렬과의 텐서 곱셈을 통하여 변환됩니다. 변환된 모든 벡터는 (height, width, output_depth) 크기의 3D 특성 맵을 재구성됩니다. 출력 특성 맵의 공간상 위치는 입력 특성 맵의 갑은 위치에 대응됩니다. 3x3 윈도우를 사용하면 3D 패치 input[i-1:i+2, j-1:j+2, :]로 부터 벡터 output [i, j, :]가 만들어 집니다.

<br></br>

![](../images/5-1-cnn4.png)
<br></br>

출력 높이와 너비는 입력의 높이, 너비와 다를 수 있습니다. 여기에는 두가지 이유가 있습니다.
* 경계 문제, 입력 특성 맵에 패딩을 추가하여 대응할 수 있습니다.
* 스트라이드 (stride)의 사용 여부에 따라 다릅니다.

#### 경계 문제와 패딩 이해하기

5x5 크기의 특성 맵을 생각해보겠습니다. (총 25개의 타일). 3x3 크기인 윈도우의 중앙을 맞출 수 있는 타일은 3x3 격자를 형성하는 9개뿐입니다. 따라서 출력 특성 맵은 3x3 크기가 됩니다. 여기서 높이와 너비 차원을 따라 정확히 2개의 타일이 줄었습니다. 

<br></br>
![](../images/5-1-cnn5.png)
<br></br>

입력과 동일한 높이와 너비를 가진 출력 특성 맵을 얻고 싶다면 **패딩(padding)**을 사용할 수 있습니다. 패딩은 입력 특성 맵의 가장자리에 적절한 개수의 행과 열을 추가합니다.

<br></br>
![](../images/5-1-cnn6.png)
<br></br>

Conv2D층에서 패딩은 `padding` 매개변수로 설정할 수 있습니다. 2개의 값이 가능합니다.
* `valid`는 패딩을 사용하지 않는다는 뜻
* `same`은 입력과 동일 높이와 너비를 가진 출력을 만들기 위해 패딩한다. 입니다.
* 기본값은 `valid`입니다.

#### 합성곱 스트라이드 이해하기

지금까지 합성곱에 대한 설명은 합성곱 윈도우의 중앙 타일이 연속적으로 지나간다고 가정한 것입니다. 두번의 연속적인 윈도우 사이의 거리가 *스트라이드*라고 불리는 합성곱의 파라미터입니다. 스트라이드가 1보다 큰 **스트라이드 합성곱**도 가능합니다. 아래는 스트라이드 2를 사용한 3x3 크기의 윈도우로 합성곱하여 추출한 패치입니다.

<br></br>
![](../images/5-1-cnn7.png)
<br></br>

스트라이드 2를 사용했다는 것은 특성 맵의 너비와 높이가 2의 배수로 다운샘플링됐다는 뜻입니다. 특성 맵을 다운샘플링하기 위해서 스트라이드 대신에 **최대 풀링 (max pooling)** 연산을 사용하는 경우가 많습니다.

### 최대 풀링 연산

앞선 예제에서는 특성 맵의 크기가 MaxPooling2D 층마다 절반으로 줄었습니다. 예를 들어 첫번째 MaxPooling2D 층 이전에 특성 맵의 크기는 26x26이었는데 최대 풀링 연산으로 13x13으로 줄었습니다. 스트라이드와 같이 강제적으로 특성 맵을 다운샘플링하는 것이 최대 풀링의 역할입니다.

최대 풀링은 입력 특성 맵에서 윈도우에 맞는 패치를 추출하고 각 채널별로 최댓값을 출력합니다. 합성곱과 개념적으로 비슷하지만 추출한 패치에 학습된 선형 변환을 적용하는 대신 **하드코딩된 최댓값 추출 연산**을 사용합니다. 합성곱과 가장 큰 차이점은 최대 풀링은 보통 2x2 윈도우와 스트라이드 2를 사용하여 특성 맵을 절반 크기로 다운샘플링한다는 것입니다. 이에 반해 합성곱은 전형적으로 3x3 윈도우와 스트라이드 1을 사용합니다.

왜 다운샘플링할까요?

In [8]:
model_no_max_pool = models.Sequential()
model_no_max_pool.add(layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (28, 28, 1)))
model_no_max_pool.add(layers.Conv2D(64, (3, 3), activation = 'relu'))
model_no_max_pool.add(layers.Conv2D(64, (3, 3), activation = 'relu'))

model_no_max_pool.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 22, 22, 64)        36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


이 설정에서 무엇이 문제일까요?
1. 특성의 공간적 계층 구조를 학습하는데 도움이 되지 않습니다. 세번째 층의 3x3 윈도우는 초기 입력의 7x7 윈도우 영역에 대한 정보만 담고 있습니다. 컨브넷에 의해 학습된 고수준 패턴은 최기 입력에 관한 정보가 아주 적어 숫자 분류를 학습하기에 충분하지 않습니다. 마지막 합성곱 층의 특성이 전체 입력에 대한 정보를 가지고 있어야 합니다.
    * 7x7 픽셀 크기의 창으로 숫자를 보고 분류해보세요!
2. 최종 특성 맵은 22 x 22 x 64 = 30976개의 가중치를 가집니다. 이 컨브넷을 펼친 후 512 크기의 Dense층과 연결한다면 약 1580만개의 가중치 파라미터가 생깁니다 작은 모델치고는 너무 많은 가중치고, 심각한 과대적합이 발생할 것입니다.

간단하게 다운샘플링을 사용하는 이유는 처리할 특성 맵의 가중치 개수를 줄이기 위해서입니다. 또한 연속적인 합성곱 층이 점점 커진 윈도우를 통해 바라보도록 만들어 필터의 공간적인 계층 구조를 구성합니다.

최대 풀링이 다운샘플링을 할 수 있는 유일한 방법은 아닙니다. 최댓값을 취하는 최대 풀링 대신에 입력 패치의 채널별 평균값을 게산하여 변환하는 **평균 풀링 (average pooling)**을 사용할 수도 있습니다. 하지만 최대 풀링이 다른 방법들보다 더 잘 작동하는 편입니다. 그 이유는 특성이 특성 맵의 각 타일에서 어떤 패턴이나 개념의 존재 여부를 인코딩하는 경향이 있기 때문입니다. (그래서 **특성 지도(맵)**이라고 불립니다). 따라서 특성의 평균값보다 여러 특성 중에서 최댓값을 사용하는 것이 더 유용합니다. 가장 납득할 만한 서브샘플링 (sub-sampling) 전략은 먼저 스트라이드가 없는 합성곱으로 조밀한 특성 맵을 만들고 그 다음 작은 패치에 대해서 최대로 활성화된 특성을 고르는 것입니다.